In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 导入包和数据

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from math import ceil
from multiprocessing import Pool       
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/'
fea_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Features/'

train_df = pd.read_csv(datasets_path + 'sales_train_evaluation.csv') # d1-d1941
prices_df = pd.read_csv(datasets_path + 'sell_prices.csv')
calendar_df = pd.read_csv(datasets_path + 'calendar.csv')

## 基本函数

In [ ]:
# 减小内存的函数
def reduce_mem_usage(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2 # memory_usage() 返回每列的内存使用情况（以字节为单位）  字节B到MB需要除以两个1024    
    for col in df.columns: 
      col_type = df[col].dtypes
      if col_type in numerics:
        c_min = df[col].min()
        c_max = df[col].max()
        if str(col_type)[:3] == 'int':
          if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
          elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
          elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
          elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)  
        else:
          if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
            df[col] = df[col].astype(np.float16)
          elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            df[col] = df[col].astype(np.float32)
          else:
            df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
      # str.format()格式化数字是通过" {} "和" : "来代替以前的" % "，5代表整个浮点数占5个字节，2代表浮点数小数点后保留2位 
    return df 

## 获取内存的大小
def get_memory_usage():
  return np.round(psutil.Process(os.getpid()).memory_info()[0] / 2.**30, 2) 
    # os.getpid()获取当前进程id 
    # psutil.Process(os.getpid())获取当前进行的相关信息，比如：psutil.Process(pid=18568, name='python.exe', started='16:07:24')
    # Process(os.getpid()).memory_info()[0]获取rss的值，rss是常驻内存集(Resident Set Size),表示该进程分配的内存大小（每一刻都在变化）。
    # rss除以2的30次方（rss为B，除以2**30，得到GB），np.round(x, 2)保留小数点2位的四舍五入

## 内存大小的单位
def sizeof_fmt(num, suffix='B'): # suffix后缀
  for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']: # 循环直到 abs(num) < 1024.0 输出
    if abs(num) < 1024.0:
      return "%3.1f%s%s" % (num, unit, suffix) 
    num /= 1024.0
  return "%.1f%s%s" % (num, 'Yi', suffix)

## dataframe的拼接
def merge_by_concat(df1, df2, merge_on):
  merged_gf = df1[merge_on]
  merged_gf = merged_gf.merge(df2, on=merge_on, how='left') 
  new_columns = [col for col in list(merged_gf) if col not in merge_on] # list(merged_gf) == merged_df.columns
  df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
  return df1

## 生成grid_part_1.pkl文件


In [ ]:
## 一些变量的定义
TARGET = 'sales'        
END_TRAIN = 1941         
MAIN_INDEX = ['id','d']  

## 将d由横展成列
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, id_vars = index_columns, var_name = 'd', value_name = TARGET) 
# id_vars为不变的列，value_vars为需要转换的列，全部需要转化，就不写了。
# 59181090 不等于 30490 * 1947 = 59364030 原因可能是有的商品在某一天的销量没有记录
print('Train rows:', len(train_df), len(grid_df)) 

## 为1941后的28天试用Nan补充
add_grid = pd.DataFrame()
for i in range(1,29):
  temp_df = train_df[index_columns]
  temp_df['d'] = 'd_'+ str(END_TRAIN+i) 
  temp_df[TARGET] = np.nan
  add_grid = pd.concat([add_grid,temp_df]) # 纵向连接，索引不变
  del temp_df # 中间变量，循环一次删除一次

grid_df = pd.concat([grid_df,add_grid],ignore_index=True) # 纵向连接，重置索引 注意：把grid_df、add_grid用列表合并起来，再作为pd.concat的参数

############ 目前的grid_df：(59181090+853720, 8) = (60034810, 8)

# 删除没用的数据，清理内存
del add_grid, train_df 
gc.collect()

print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum()))) # >20 "＞"表示右对齐(宽度为20)
#    Original grid_df:   3.6GiB 宽度为20，Original grid_df只占16，前面还空4位

for col in index_columns:
  grid_df[col] = grid_df[col].astype('category') # 前六列转化为category类型

print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum()))) 
#     Reduced grid_df:   1.3GiB
###############原来前六列的数据类型为object(str或mixed)，记住：object的数据类型占用内存是最大，转化为category类型的

## 制作release特征
print('Release week')
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index() # groupby按行聚合，agg按列聚合  最后生成一列 min 
# 按store_id和item_id这两列groupby再取wm_yr_wk这一列，就得到，比如('WI_3', 'HOUSEHOLD_2_476') 6503151  11313  6503152  11314 ... 
# 在满足('WI_3', 'HOUSEHOLD_2_476')的条件下，对应原索引的wm_yr_wk这一Series，然后每一Series再agg取最小值
# 目的是要得到10个商店每个商品最早开始销售的周数
release_df.columns = ['store_id','item_id','release'] # 把 "min" 列改名为 "release" 列

grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id']) # grid_df 增加一个特征 release 变9列
del release_df 

grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d']) # 增加 wm_yr_wk 特征，10列
             
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']] # 由60034810行变成47735397行 
# 这句代码的意义：有的产品上架日期要比calendar里面的日期要晚，对于这些数据是没有意义的，直接剔除即可。
# 比如，商品HOBBIES_1_001在11325周上架(上架才有销量)，而数据是在11101周开始记录的。这部分数据让机器学习的话，是没有意义的。
grid_df = grid_df.reset_index(drop=True) 

############## 此时，grid_df为(47735397, 10)

print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

print('Save Part 1')

grid_df.to_pickle(pkl_path + 'grid_part_1.pkl') # 前六 + d、sales、release和wm_yr_wk 

print('Size:', grid_df.shape) # Size: (47735397, 10)

Train rows: 30490 59181090
    Original grid_df:   3.6GiB
     Reduced grid_df:   1.3GiB
Release week
    Original grid_df:   1.8GiB
     Reduced grid_df:   1.5GiB
Save Part 1
Size: (47735397, 10)


## 生成grid_part_2.pkl文件（价格特征）

In [ ]:
prices_df.groupby(['store_id','item_id'])['sell_price'].describe()

count       mean           std  ...    50%    75%    max
store_id item_id                                          ...                     
CA_1     FOODS_1_001      282.0   2.169362  1.095719e-01  ...   2.24   2.24   2.24
         FOODS_1_002      282.0   8.937447  4.860669e-01  ...   8.88   9.48   9.48
         FOODS_1_003      282.0   2.976809  1.568385e-01  ...   2.88   3.23   3.23
         FOODS_1_004      225.0   1.851378  1.032610e-01  ...   1.78   1.96   1.96
         FOODS_1_005      282.0   3.334468  1.796890e-01  ...   3.28   3.54   3.54
...                         ...        ...           ...  ...    ...    ...    ...
WI_3     HOUSEHOLD_2_512  282.0   3.970000  2.046442e-14  ...   3.97   3.97   3.97
         HOUSEHOLD_2_513  168.0   2.780000  1.158084e-14  ...   2.78   2.78   2.78
         HOUSEHOLD_2_514  282.0  18.805603  5.890228e-01  ...  18.97  18.97  19.54
         HOUSEHOLD_2_515  127.0   1.970000  8.916960e-16  ...   1.97   1.97   1.97
         HOUSEHOLD_2_516  282.0   5.940000  3.648005e-14  ...   5.94   5.94   5.94

[30490 rows x 8 columns]

In [ ]:
print('Prices')

## 价格的统计特性
price_grouped = prices_df.groupby(['store_id','item_id'])['sell_price'] 
# price_grouped 是一个SeriesGroupBy object，该对象可调用describe()方法查看各组统计数据，即 price_grouped.describe()

prices_df['price_max'] = price_grouped.transform('max') # 'max' == np.max 这里相当于取每组内的最大值，生成一个新列
prices_df['price_min'] = price_grouped.transform('min') # 如果需要自己的写函数的话，可以用匿名函数，比如lambda x: x * x
prices_df['price_std'] = price_grouped.transform('std') 
prices_df['price_mean'] = price_grouped.transform('mean') 
prices_df['price_norm'] = prices_df['sell_price'] / prices_df['price_max'] # 归一化，每个商品除以它的最大值
prices_df['price_nunique'] = price_grouped.transform('nunique') # nunique 每组内不同值的个数
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique') # 同一商店内相同价位的不同商品个数

## 价格的商店、商品等做的统计特征
calendar_prices = calendar_df[['wm_yr_wk','month','year']] # (1969, 3)
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk']) # 只重复的只保留第一个 (282, 3) 一周7天，去掉重复的6天
# prices_df (6841121, 11) 4+7
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
# prices_df (6841121, 13) 4+7+2
del calendar_prices

prices_df['price_momentum'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
# 本周的售价 / 上一周的售价 = 商品售价的变化率（势头momentum）
prices_df['price_momentum_m'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
# 同一商店同一商品的售价 / 同一商店同一商品在同一个月售价均值的变化率
prices_df['price_momentum_y'] = prices_df['sell_price'] / prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')
# 同一商店同一商品的售价 / 同一商店同一商品在同一个年售价均值的变化率
del prices_df['month'], prices_df['year'] # 可以直接通过del删除dataframe的某一列

# prices_df 4(原)+7(价格的统计特征)+2+3(变化率)-2 = 14 

print('Merge prices and save part 2')

original_columns = list(grid_df) # 前六 + d、sales、release和wm_yr_wk 共10

grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left') # 特征数：10 + 14 - 3(重叠) = 21
keep_columns = [col for col in list(grid_df) if col not in original_columns] # 14 - 3 = 11
grid_df = grid_df[MAIN_INDEX + keep_columns] 
# MAIN_INDEX ：'id', 'd' 
# keep_columns ：'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 
#         'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y'
grid_df = reduce_mem_usage(grid_df) # 62.2% reduction Size: (47735397, 13)
grid_df.to_pickle(pkl_path + 'grid_part_2.pkl')
del prices_df

Prices
Merge prices and save part 2
Memory usage decreased to 1822.44 Mb (62.2% reduction)


## 生成grid_part_3.pkl文件（美国节假日、日期特征）

In [ ]:
grid_df = pd.read_pickle(pkl_path + 'grid_part_1.pkl')
grid_df = grid_df[MAIN_INDEX]
icols = ['date', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left') # (47735397, 2+8)
#################################################### add_holiday #####################################################################
df_holidays = pd.read_csv(fea_path + 'usholidays.csv') # 美国的从 1966-12-25 到 2020-12-25 所有节假日，有 Date 和 Holiday 两列
df_holidays.drop(columns = 'Unnamed: 0', inplace=True) 
df_holidays.rename(columns = {'Date':'date'}, inplace=True)  
grid_df = grid_df.merge(df_holidays, on=['date'], how='left') # (47735397, 2+8+1)
#########################################################################################################################################
# 把'snap_' 列转化为 bool or int8  减少数据内存
icols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'Holiday']
for col in icols:
  grid_df[col] = grid_df[col].astype('category') 

# 把 date 日期列转化为 DateTime 类型
grid_df['date'] = pd.to_datetime(grid_df['date'])

# 从date中抽取一些日期特征，都是int64，可以转化为int8或者16
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8) 
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8) # 这个日期在一年里面是第几周，比如2011/1/29是第四周
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8) 
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8) # min：2011,max：2016 变成0-5
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8) 
# ceil()向上舍入最接近的整数值，比如day=2，ceil(day/7) = 1，目的是确定这一天在属于这个月的第几周，如果day=29、30、31，则ceil(day/7)=5，即第五周
grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8) # 这一天属于本周的第几天，比如周六——5，周日——6，周一——0
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8) # 把布尔转化为1，0  周六、周日为1，工作日为0
del grid_df['date']

print('Save part 3')
grid_df.to_pickle(pkl_path + 'grid_part_3.pkl')  # Size: (47735397, 17)
del calendar_df
del grid_df

grid_df = pd.read_pickle(pkl_path + 'grid_part_1.pkl') # (47735397, 10)
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16) # 比如d_1，去掉"d_"只剩下1
del grid_df['wm_yr_wk'] # (47735397, 9)
grid_df.to_pickle(pkl_path + 'grid_part_1.pkl')
del grid_df
gc.collect()

Save part 3
Index(['id', 'd', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'Holiday', 'tm_d',
       'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end'],
      dtype='object')
(47735397, 17)


## grid_part_4.pkl(lag and rolling features)

In [ ]:
import time

grid_df = pd.read_pickle(pkl_path + 'grid_part_1.pkl')[['id','d','sales']] # (47735397, 3)

SHIFT_DAY = 28
start_time_1 = time.time() 
print('Create lags')

LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)] # 28-42

grid_df = grid_df.assign(**{'{}_lag_{}'.format(TARGET, day): grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(day)) for day in LAG_DAYS})
# DataFrame.assign(**kwargs)为DataFrame分配新列。参考：https://www.cjavapy.com/article/318/
# 由于要添加多个列，需要传入字典，键名为生成的列名，键值为对应的Series
# 解释：sales_lag_28，某商店中的某产品这一天的销量把28天前的销量当作特征，当作一个参考。
# 比如，预测某店铺某商品的1942的销量参考了它28天前(1914)的销量，
#             1943的销量参考了它28天前(1915)、29天前(1914)的销量，
#             1944的销量参考了它28天前(1916)、29天前(1915)、30天前(1914)的销量，
#             1945的销量参考了它28天前(1917)、29天前(1916)、30天前(1915)、31天前(1914)的销量，...
# 每个商品某一天的销量都可以把它42天前、41天前、...、28天前(15天)的销量作为训练的特征，用来作为参考。
for col in list(grid_df):
  if 'lag' in col:
    grid_df[col] = grid_df[col].astype(np.float16) # 把lag这14列，数据类型转化为float16
# grid_df (47735397, 3+15=18) sales_lag_28 ... sales_lag_42(15个)
print('%0.2f min: Lags' % ((time.time() - start_time_1) / 60)) # (time.time()：当前时刻的时间(s)


print('Create rolling aggs')
start_time_2 = time.time()

for i in [7,14,30,60,180]: # i 是选择窗口的大小
  print('Rolling period:', i)
  grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
  grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16) 
# 滑动时间窗法可以使时序更加平滑。先滞后，再平滑，会导致前面存在一部分NAN值。
# 比如x.shift(28)得到每个商品Series前28个元素均为NAN，再对Series.rolling(7).mean()，因为时间窗滑动的过程中，如果存在NAN，取均值也是NAN。
# 故会有  SHIFT_DAY + i(时间窗大小) -1  个NAN值
# 得到 2 * 5 = 10 列 grid_df (47735397, 18 + 10 = 28)
for d_shift in [1,7,14]: 
  print('Shifting period:', d_shift)
  for d_window in [7,14,30,60]:
    col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
    grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16) 
# 得到 3 * 4 = 12 列 grid_df (47735397, 28 + 12 = 40)
# 注：时间窗能更好的体现时序中某个点的数据
print('%0.2f min: Lags' % ((time.time() - start_time_2) / 60))
print('Save lags and rollings')

grid_df.to_pickle(pkl_path + 'grid_part_4.pkl') 
del grid_df

Create lags
9.32 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
21.04 min: Lags
Save lags and rollings
Index(['id', 'd', 'sales', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30',
       'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34',
       'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38',
       'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42',
       'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14',
       'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60',
       'rolling_std_60', 'rolling_mean_180', 'rolling_std_180',
       'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
       'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60',
       'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14',
       'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60',
       'rolling_mean_tmp_14_7', 'rolling_

## grid_part_5.pkl(对sales按层聚合求mean/std features)

In [ ]:
grid_df = pd.read_pickle(pkl_path + 'grid_part_1.pkl')
base_cols = list(grid_df) # Size：(47735397, 9)
# base_cols：['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'release']

icols = [['state_id'], # 三个州 的销量聚合之后得到3组，分别求mean、std
      ['store_id'], # 十个商店 的销量聚合之后得到10组，分别求mean、std
      ['cat_id'], # 三个类别 的销量聚合之后得到3组，分别求mean、std
      ['dept_id'], # 七个部门 的销量聚合之后得到7组，分别求mean、std
      ['state_id', 'cat_id'], # CA(三个类别)、TX(三个类别)、WI(三个类别) 的销量聚合之后得到9组，分别求mean、std
      ['state_id', 'dept_id'], # CA(七个部门)、TX(七个部门)、WI(七个部门) 的销量聚合之后得到21组，分别求mean、std
      ['store_id', 'cat_id'], # CA1——CA4(三个类别)、TX1——TX3(三个类别)、WI1——WI3(三个类别) 的销量聚合之后得到30组，分别求mean、std
      ['store_id', 'dept_id'], # CA1——CA4(七个部门)、TX1——TX3(七个部门)、WI1——WI3(七个部门) 的销量聚合之后得到70组，分别求mean、std
      ['item_id'], # 3049个商品 的销量聚合之后得到3049组(每组十个)，分别求mean、std
      ['item_id', 'state_id'], # CA(3049个商品)聚合得到3049组(每组4个)、TX(3049个商品)聚合得到3049组(每组3个)、WI(3049个商品)聚合得到3049组(每组3个)
      ['item_id', 'store_id']] # 十个商店(每个商店3049个商品) 的销量聚合之后得到30490组(每组一个)
      ## 另外还有一个所有州的销量加到一起，然后取mean、std
for col in icols:
  print('Encoding', col)
  col_name = '_' + '_'.join(col) + '_' # str.join(sequence) 返回sequence中的字符串按照str连接起来的字符串，
  # 比如'_'.join(['store_id', 'cat_id']) 得到 'store_id_cat_id' 中间的 '_' 就是连接 'store_id' 和 'cat_id' 的str
  grid_df['enc' + col_name + 'mean'] = grid_df.groupby(col)['sales'].transform('mean').astype(np.float16)
  grid_df['enc' + col_name + 'std'] = grid_df.groupby(col)['sales'].transform('std').astype(np.float16)

## grid_df 特征数：11 * 2 + 9 = 31

keep_cols = [col for col in list(grid_df) if col not in base_cols] # 11 * 2 
grid_df = grid_df[['id','d'] + keep_cols] # 2 + 11 * 2 = 24

print('Save Mean/Std encoding')
grid_df.to_pickle(pkl_path + 'grid_part_5.pkl')
print(grid_df.columns)
print(grid_df.shape)
del grid_df

Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']
Save Mean/Std encoding
Index(['id', 'd', 'enc_state_id_mean', 'enc_state_id_std', 'enc_store_id_mean',
       'enc_store_id_std', 'enc_cat_id_mean', 'enc_cat_id_std',
       'enc_dept_id_mean', 'enc_dept_id_std', 'enc_state_id_cat_id_mean',
       'enc_state_id_cat_id_std', 'enc_state_id_dept_id_mean',
       'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
       'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
       'enc_store_id_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
       'enc_item_id_state_id_mean', 'enc_item_id_state_id_std',
       'enc_item_id_store_id_mean', 'enc_item_id_store_id_std'],
      dtype='object')
(47735397, 24)


## CA1_TX2_TX3 holidays features

In [ ]:
calendar_win = calendar_df[['event_name_1','d']]
calendar_win['event_name_1_win'] = np.nan # 假日效应列
## 超级碗
SuperBowl_d = [9, 373, 737, 1101, 1465, 1836]
for i in SuperBowl_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'SuperBowl1'
    d_2['event_name_1_win'] = 'SuperBowl1'
    d_3['event_name_1_win'] = 'SuperBowl2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_3


## 情人节，提前一天
ValentinesDay_d = [17, 382, 748, 1113, 1478, 1843]
for i in ValentinesDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'ValentinesDay'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

### LentStart贷款开始日，前一天跟后一天貌似都比较低
LentStart_d = [40, 390, 747, 1132, 1482, 1839]
for i in LentStart_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'LentStart1'
    d_2['event_name_1_win'] = 'LentStart2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## LentWeek2贷款周，也是周三的
LentWeek_d = [47, 397, 754, 1139, 1489, 1846]
for i in LentWeek_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'LentWeek1'
    d_2['event_name_1_win'] = 'LentWeek2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 复活节
Ester_d = [86, 436, 793, 1178, 1528, 1885]
for i in Ester_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Ester1'
    d_2['event_name_1_win'] = 'Ester1'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

## Mother's day(Cultural) 
Mother_d = [100, 471, 835, 1199, 1563, 1927]
for i in Mother_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Mother_day1'
    d_2['event_name_1_win'] = 'Mother_day1'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

## MemorialDay(National),放假，对前一天和后一天都会有影响的
MemorialDay_d = [122, 486, 850, 1214, 1578, 1949]
for i in MemorialDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'MemorialDay1'
    d_2['event_name_1_win'] = 'MemorialDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 父亲节
Father_d = [142, 506, 870, 1234, 1605, 1969]
for i in Father_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Father1'
    d_2['event_name_1_win'] = 'Father1'
    d_3['event_name_1_win'] = 'Father2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_3

## 独立日,前后一天
IndependenceDay_d = [157, 523, 888, 1253, 1618]
for i in IndependenceDay_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'IndependenceDay1'
    d_2['event_name_1_win'] = 'IndependenceDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 劳动节，放假1天
LaborDay_d = [220, 584, 948, 1312, 1683]
for i in LaborDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'LaborDay1'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

## ColumbusDay(National) 哥伦布日 放假1天
ColumbusDay_d = [255, 619, 990, 1354, 1718]
for i in ColumbusDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'ColumbusDay1'
    d_2['event_name_1_win'] = 'ColumbusDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## Halloween，万圣节前夕
Halloween_d = [276, 642, 1007, 1372, 1737]
for i in Halloween_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Halloween1'
    d_2['event_name_1_win'] = 'Halloween2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

# EidAlAdha 
EidAlAdha_d = [283, 637, 991, 1345, 1700]
for i in EidAlAdha_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'EidAlAdha1'
    d_2['event_name_1_win'] = 'EidAlAdha2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## VeteransDay 放假一天，光明节替换的
VeteransDay_d = [287, 653, 1018, 1383, 1748]
for i in VeteransDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'VeteransDay1'
    d_2['event_name_1_win'] = 'VeteransDay1'
    d_3['event_name_1_win'] = 'VeteransDay2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_3

# Thanksgiving 感恩节 	美国人合家欢聚的节日  放假1天
Thanksgiving_d = [300, 664, 1035, 1399, 1763]
for i in Thanksgiving_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Thanksgiving1'
    d_2['event_name_1_win'] = 'Thanksgiving1'
    d_3['event_name_1_win'] = 'Thanksgiving2'
    d_4['event_name_1_win'] = 'Thanksgiving3'

    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4

# 圣诞节
Christmas_d = [331, 697, 1062, 1427, 1792]
for i in Christmas_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-6}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-5}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-4}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_7 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_8 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_9 = calendar_win[calendar_win['d']==f'd_{i+3}']
    d_10 = calendar_win[calendar_win['d']==f'd_{i+4}']
    d_11 = calendar_win[calendar_win['d']==f'd_{i+5}']
    d_12 = calendar_win[calendar_win['d']==f'd_{i+6}']

    d_1['event_name_1_win'] = 'Christmas1'
    d_2['event_name_1_win'] = 'Christmas1'
    d_3['event_name_1_win'] = 'Christmas1'
    d_4['event_name_1_win'] = 'Christmas1'
    d_5['event_name_1_win'] = 'Christmas1'
    d_6['event_name_1_win'] = 'Christmas1'

    d_7['event_name_1_win'] = 'Christmas2'
    d_8['event_name_1_win'] = 'Christmas2'
    d_9['event_name_1_win'] = 'Christmas2'
    d_10['event_name_1_win'] = 'Christmas2'
    d_11['event_name_1_win'] = 'Christmas2'
    d_12['event_name_1_win'] = 'Christmas2'

    calendar_win[calendar_win['d']==f'd_{i-6}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-5}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-4}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_6

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_7
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_8
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_9
    calendar_win[calendar_win['d']==f'd_{i+4}'] = d_10
    calendar_win[calendar_win['d']==f'd_{i+5}'] = d_11
    calendar_win[calendar_win['d']==f'd_{i+6}'] = d_12

## NewYear，圣诞节前1周开始放假，一直放到元旦之后的周一
NewYear_d = [338, 704, 1069, 1434, 1799]
for i in NewYear_d:

    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i+3}']
    d_7 = calendar_win[calendar_win['d']==f'd_{i+4}']
    d_8 = calendar_win[calendar_win['d']==f'd_{i+5}']
    d_9 = calendar_win[calendar_win['d']==f'd_{i+6}']

    # 接下来几天的节假日效应相同，都用NewYear2表示
    d_4['event_name_1_win'] = 'NewYear2' 
    d_5['event_name_1_win'] = 'NewYear2'
    d_6['event_name_1_win'] = 'NewYear2'
    d_7['event_name_1_win'] = 'NewYear2'
    d_8['event_name_1_win'] = 'NewYear2'
    d_9['event_name_1_win'] = 'NewYear2'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_6
    calendar_win[calendar_win['d']==f'd_{i+4}'] = d_7
    calendar_win[calendar_win['d']==f'd_{i+5}'] = d_8
    calendar_win[calendar_win['d']==f'd_{i+6}'] = d_9

calendar_win.drop(columns=['event_name_1'],inplace=True)
## 改一下名字
calendar_win.to_csv(fea_path + 'CA1_TX2_TX3_holidays.csv',index=False) # ['d', 'event_name_1_win'] (1969, 2)
del calendar_win

## CA2 holidays features

In [ ]:
calendar_win = calendar_df[['event_name_1','d']]
calendar_win['event_name_1_win'] = np.nan # 假日效应列

### 超级碗
SuperBowl_d = [9, 373, 737, 1101, 1465, 1836]

for i in SuperBowl_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'SuperBowl1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

## 情人节，提前一天
ValentinesDay_d = [17, 382, 748, 1113, 1478, 1843]
for i in ValentinesDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'ValentinesDay1'
    d_2['event_name_1_win'] = 'ValentinesDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## LentWeek2贷款周，也是周三的
LentWeek_d = [47, 397, 754, 1139, 1489, 1846]
for i in LentWeek_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'LentWeek1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
 
Mother_d = [100, 471, 835, 1199, 1563, 1927]
for i in Mother_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Mother_day1'
    d_2['event_name_1_win'] = 'Mother_day2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

## MemorialDay(National),放假，对前一天和后一天都会有影响的
MemorialDay_d = [122, 486, 850, 1214, 1578, 1949]
for i in MemorialDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'MemorialDay1'
    d_2['event_name_1_win'] = 'MemorialDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 父亲节
Father_d = [142, 506, 870, 1234, 1605, 1969]
for i in Father_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Father1'
    d_2['event_name_1_win'] = 'Father1'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

## 独立日,前后一天
IndependenceDay_d = [157, 523, 888, 1253, 1618]
for i in IndependenceDay_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'IndependenceDay1'
    d_2['event_name_1_win'] = 'IndependenceDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 劳动节，放假1天

LaborDay_d = [220, 584, 948, 1312, 1683]
for i in LaborDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'LaborDay2'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

## ColumbusDay(National) 哥伦布日 放假1天
ColumbusDay_d = [255, 619, 990, 1354, 1718]
for i in ColumbusDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'ColumbusDay1'
    d_2['event_name_1_win'] = 'ColumbusDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## Halloween，万圣节前夕
Halloween_d = [276, 642, 1007, 1372, 1737]
for i in Halloween_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Halloween1'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

## VeteransDay ,放假一天
VeteransDay_d = [287, 653, 1018, 1383, 1748]
for i in VeteransDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'VeteransDay1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

# Thanksgiving 感恩节 	美国人合家欢聚的节日  放假1天
Thanksgiving_d = [300, 664, 1035, 1399, 1763]
for i in Thanksgiving_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Thanksgiving1'
    d_2['event_name_1_win'] = 'Thanksgiving2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

# 圣诞节
Christmas_d = [331, 697, 1062, 1427, 1792]
for i in Christmas_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i+3}']


    d_1['event_name_1_win'] = 'Christmas1'
    d_2['event_name_1_win'] = 'Christmas1'

    d_3['event_name_1_win'] = 'Christmas2'

    d_4['event_name_1_win'] = 'Christmas3'

    d_5['event_name_1_win'] = 'Christmas4'
    d_6['event_name_1_win'] = 'Christmas4'

    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_6

## NewYear,圣诞节前1周开始放假，一直放到元旦之后的周一
NewYear_d = [338, 704, 1069, 1434, 1799]
for i in NewYear_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i+3}']

    d_1['event_name_1_win'] = 'NewYear1'
    d_2['event_name_1_win'] = 'NewYear1'
    d_3['event_name_1_win'] = 'NewYear2'
    d_4['event_name_1_win'] = 'NewYear3'
    d_5['event_name_1_win'] = 'NewYear4'
    d_6['event_name_1_win'] = 'NewYear4'

    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_6
  
## 复活节
Ester_d = [86, 436, 793, 1178, 1528, 1885]
for i in Ester_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Ester1'
    d_2['event_name_1_win'] = 'Ester2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

# MartinLutherKingDay:
MartinLutherKingDay_d = [353, 724, 1088, 1452, 1816]
for i in MartinLutherKingDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'MartinLutherKingDay'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

PresidentsDay_d = [24, 388, 752, 1116, 1480, 1844]
for i in PresidentsDay_d:

    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_2['event_name_1_win'] = 'PresidentsDay_1'
    d_3['event_name_1_win'] = 'PresidentsDay_2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_3

calendar_win.drop(columns=['event_name_1'],inplace=True)
calendar_win.to_csv(fea_path + 'CA2_holidays.csv',index=False) # ['d', 'event_name_1_win'] (1969, 2)
del calendar_win

           d event_name_1_win
0        d_1              NaN
1        d_2              NaN
2        d_3              NaN
3        d_4              NaN
4        d_5              NaN
...      ...              ...
1964  d_1965              NaN
1965  d_1966              NaN
1966  d_1967          Father1
1967  d_1968          Father1
1968  d_1969              NaN

[1969 rows x 2 columns]


## WI3 holidays features

In [ ]:
calendar_win = calendar_df[['event_name_1','d']]
calendar_win['event_name_1_win'] = np.nan # 假日效应列

### 超级碗
SuperBowl_d = [9, 373, 737, 1101, 1465, 1836]
for i in SuperBowl_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}'] # 得到一个 (1, 3) 的dataframe

    d_1['event_name_1_win'] = 'SuperBowl1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

## 情人节
ValentinesDay_d = [17, 382, 748, 1113, 1478, 1843]
for i in ValentinesDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'ValentinesDay1'
    d_2['event_name_1_win'] = 'ValentinesDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## LentWeek2
LentWeek_d = [47, 397, 754, 1139, 1489, 1846]
for i in LentWeek_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'LentWeek1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
 
Mother_d = [100, 471, 835, 1199, 1563, 1927]
for i in Mother_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Mother_day1'
    d_2['event_name_1_win'] = 'Mother_day2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## MemorialDay(National),放假，对前一天和后一天都会有影响的
MemorialDay_d = [122, 486, 850, 1214, 1578, 1949]
for i in MemorialDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']


    d_1['event_name_1_win'] = 'MemorialDay1'
    d_2['event_name_1_win'] = 'MemorialDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 父亲节
Father_d = [142, 506, 870, 1234, 1605, 1969]
for i in Father_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Father1'
    d_2['event_name_1_win'] = 'Father1'
    d_3['event_name_1_win'] = 'Father2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_3

## 独立日,前后一天
IndependenceDay_d = [157, 523, 888, 1253, 1618]
for i in IndependenceDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'IndependenceDay1'
    d_2['event_name_1_win'] = 'IndependenceDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## 劳动节，放假1天
LaborDay_d = [220, 584, 948, 1312, 1683]
for i in LaborDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'LaborDay2'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

## ColumbusDay(National) 哥伦布日 放假1天

ColumbusDay_d = [255, 619, 990, 1354, 1718]
for i in ColumbusDay_d:
    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'ColumbusDay1'
    d_2['event_name_1_win'] = 'ColumbusDay2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2

## Halloween，万圣节前夕
Halloween_d = [276, 642, 1007, 1372, 1737]
for i in Halloween_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Halloween1'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

## VeteransDay ,放假一天
VeteransDay_d = [287, 653, 1018, 1383, 1748]
for i in VeteransDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'VeteransDay1'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

# Thanksgiving 感恩节 	美国人合家欢聚的节日  放假1天
Thanksgiving_d = [300, 664, 1035, 1399, 1763]
for i in Thanksgiving_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Thanksgiving1'
    d_2['event_name_1_win'] = 'Thanksgiving2'

    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_2

# 圣诞节，滞后3天，不会跟新年效应重合 比如：1428、1429、1430 紧接着就是新年效应 1431、1432、1433
Christmas_d = [331, 697, 1062, 1427, 1792]
for i in Christmas_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i+3}']

    d_1['event_name_1_win'] = 'Christmas1'
    d_2['event_name_1_win'] = 'Christmas1'
    d_3['event_name_1_win'] = 'Christmas2'
    d_4['event_name_1_win'] = 'Christmas3'
    d_5['event_name_1_win'] = 'Christmas4'
    d_6['event_name_1_win'] = 'Christmas4'
    # 112344的原因是，离节日最近的一天效应是比较强的
    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_6

## NewYear，往前3天，不会跟圣诞节效应重合 比如：1431、1432、1433
NewYear_d = [338, 704, 1069, 1434, 1799]
for i in NewYear_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-3}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i-2}']
    d_3 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_4 = calendar_win[calendar_win['d']==f'd_{i+1}']
    d_5 = calendar_win[calendar_win['d']==f'd_{i+2}']
    d_6 = calendar_win[calendar_win['d']==f'd_{i+3}']

    d_1['event_name_1_win'] = 'NewYear1'
    d_2['event_name_1_win'] = 'NewYear1'
    d_3['event_name_1_win'] = 'NewYear2'
    d_4['event_name_1_win'] = 'NewYear3'
    d_5['event_name_1_win'] = 'NewYear4'
    d_6['event_name_1_win'] = 'NewYear4'
    # 112344的原因是，离节日最近的一天效应是比较强的  
    calendar_win[calendar_win['d']==f'd_{i-3}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i-2}'] = d_2
    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_3
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_4
    calendar_win[calendar_win['d']==f'd_{i+2}'] = d_5
    calendar_win[calendar_win['d']==f'd_{i+3}'] = d_6
  
## 复活节
Ester_d = [86, 436, 793, 1178, 1528, 1885]
for i in Ester_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']
    d_2 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'Ester1'
    d_2['event_name_1_win'] = 'Ester2'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1
    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_2


# MartinLutherKingDay:
MartinLutherKingDay_d = [353, 724, 1088, 1452, 1816]
for i in MartinLutherKingDay_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'MartinLutherKingDay'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

# Cinco De Mayo:
Mayo_d = [97, 463, 1193, 1558, 1924]
for i in Mayo_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Mayo'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

# OrthodoxChristmas:
OrthodoxChristmas_d = [344, 710, 1075, 1440, 1805]
for i in OrthodoxChristmas_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i+1}']

    d_1['event_name_1_win'] = 'OrthodoxChristmas'

    calendar_win[calendar_win['d']==f'd_{i+1}'] = d_1

# 后面添加的
OrthodoxEaster_d = [443, 828, 1535, 1920]
for i in OrthodoxEaster_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'OrthodoxEaster'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

# 后面添加的
Purim_d = [51, 405, 758, 1143, 1497, 1882]
for i in Purim_d:

    d_1 = calendar_win[calendar_win['d']==f'd_{i-1}']

    d_1['event_name_1_win'] = 'Purim'

    calendar_win[calendar_win['d']==f'd_{i-1}'] = d_1

calendar_win.drop(columns=['event_name_1'],inplace=True)
calendar_win.to_csv(fea_path + 'WI_3_holidays.csv',index=False) # ['d', 'event_name_1_win'] (1969, 2)
del calendar_win